In [171]:
import pandas as pd
import os
import time
import cv2 as cv
import numpy as np
from shapely.geometry import Polygon
from shutil import copyfile
from IPython.display import clear_output

In [2]:
all_imgs = os.listdir("test_data_10000")
bb_df = pd.read_csv("../bounding_box_data.csv", sep=",")
bb_df.head(5)

,deviceID,timestamp,x,y,width,height
0,0a:92:5e:29:ec:a2,20191225T000100+0900,660,388,125,125
1,0a:92:5e:29:ec:a2,20191225T000606+0900,660,388,125,125
2,0a:92:5e:29:ec:a2,20191225T001111+0900,660,388,125,125
3,0a:92:5e:29:ec:a2,20191225T001617+0900,660,388,125,125
4,0a:92:5e:29:ec:a2,20191225T002120+0900,660,388,125,125


In [3]:
all_filtered_cons = []
for i in range(len(all_imgs)):
    if not all_imgs[i].endswith(".png"):
        continue
    img = cv.imread("test_data_10000/" + all_imgs[i], cv.COLOR_BGR2GRAY)

    thresh = cv.adaptiveThreshold(src=img, maxValue=255, adaptiveMethod=cv.ADAPTIVE_THRESH_GAUSSIAN_C, thresholdType=cv.THRESH_BINARY_INV, blockSize=1001, C=5)
    thresh = thresh.astype(np.uint8)

    # Find countours
    contours, hierarchy = cv.findContours(image=thresh, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_NONE)[-2:]

    # Filter contours by size
    filtered_contours = []
    for con in contours:
        hull = cv.convexHull(con)
        hull_area = cv.contourArea(hull)
        con_area = cv.contourArea(con)
        x, y, w, h = cv.boundingRect(con)

        # Continue if hull area is too small or too large
        if hull_area > 40000 or hull_area < 11000:
            continue
        # Continue if hull is too close to image borders
        if (x == 0 or x > 960) or (y == 0 or y > 500):
            continue
        
        solidity = con_area / hull_area
        aspect_ratio = w / h
        if aspect_ratio > 0.8 and aspect_ratio < 1.25 and solidity > 0.5:
            filtered_contours.append(con)
    
    print("{i}\t: {filt_cons} out of {all_cons} contours passed filtering".format(i=i, filt_cons=len(filtered_contours), all_cons=len(contours)))
    all_filtered_cons.append([filtered_contours, all_imgs[i]])
    # for con in filtered_contours:
    #     x, y, w, h = cv.boundingRect(con)
    #     cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    #     cv.putText(img, "WEIGHER?", (x, y-8), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2, cv.LINE_AA)

    # cv.imshow("Bounding Boxes", img)
    # k = cv.waitKey(0)
    # cv.destroyWindow("Bounding Boxes")
    # cv.waitKey(1)
    # if k == ord("q"):
    #     break
    

0	: 0 out of 4050 contours passed filtering
1	: 0 out of 3227 contours passed filtering
2	: 0 out of 2301 contours passed filtering
3	: 0 out of 1687 contours passed filtering
4	: 0 out of 3053 contours passed filtering
5	: 0 out of 2887 contours passed filtering
6	: 1 out of 3788 contours passed filtering
7	: 0 out of 2073 contours passed filtering
8	: 0 out of 2835 contours passed filtering
9	: 0 out of 2419 contours passed filtering
10	: 0 out of 2866 contours passed filtering
11	: 0 out of 2458 contours passed filtering
12	: 0 out of 2134 contours passed filtering
13	: 0 out of 2901 contours passed filtering
14	: 0 out of 2365 contours passed filtering
15	: 1 out of 2262 contours passed filtering
16	: 0 out of 2902 contours passed filtering
17	: 0 out of 2842 contours passed filtering
18	: 1 out of 4067 contours passed filtering
19	: 1 out of 3227 contours passed filtering
20	: 0 out of 2768 contours passed filtering
21	: 0 out of 3972 contours passed filtering
22	: 0 out of 2813 c

In [4]:
# Bounding boxes supplied in format: [x, y, width, height]
def calc_iou(bbA, bbB):                   #top L     top R     bottom R   bottom L
    # Bounding boxes converted to format: [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
    bbA = Polygon([[bbA[0], bbA[1]], [bbA[0]+bbA[2], bbA[1]], [bbA[0]+bbA[2], bbA[1]-bbA[3]], [bbA[0], bbA[1]-bbA[3]]])
    bbB = Polygon([[bbB[0], bbB[1]], [bbB[0]+bbB[2], bbB[1]], [bbB[0]+bbB[2], bbB[1]-bbB[3]], [bbB[0], bbB[1]-bbB[3]]])
    
    iou = bbA.intersection(bbB).area / bbA.union(bbB).area
    return iou

In [163]:
gtf1 = os.listdir("../advanced_model/YOLOv5 Master/test/labels/")
gtf2 = os.listdir("../advanced_model/YOLOv5 Master/train/labels/")
gtf3 = os.listdir("../advanced_model/YOLOv5 Master/valid/labels/")

for i in range(len(gtf1)):
    gtf1[i] = "test/labels/" + gtf1[i]

for i in range(len(gtf2)):
    gtf2[i] = "train/labels/" + gtf2[i]

for i in range(len(gtf3)):
    gtf3[i] = "valid/labels/" + gtf3[i]

gt_files = gtf1 + gtf2 + gtf3


In [181]:
ious = []
no_pred = 0
for i in range(len(all_filtered_cons)):
    # Continue if list of predictions is empty
    if len(all_filtered_cons[i][0]) == 0:
        no_pred += 1
        continue

    time_stamp = all_filtered_cons[i][1].split("_")[0].split("+")[0] + "-0900"

    # Serch through ground truth files for the file beloning to current predictions
    for gtf in gt_files:
        if time_stamp in gtf:
            truth_file = gtf
            break
    # Test if a ground truth file could be found
    try:
        print(truth_file, end="\r")
    except:
        continue

    # Read annotation file with ground truth BB location
    f = open("../advanced_model/YOLOv5 Master/" + truth_file, "r")
    contents = f.readlines()
    f.close()
    del truth_file

    # Get ground truth BB values
    gtxc, gtyc, gtw, gth = [round(float(num)*1280) for num in contents[0].split()[1:5]]
    gtyc = round((gtyc/1280)*720)
    gtx = gtxc - round((gtw/2))
    gty = gtyc - round((((gth/1280)*720)/2))
    gt = [gtx, gty, gtw, gth]

    # If the number of bounding boxes is higher than  one loop over them to calculate IoU
    if len(all_filtered_cons[i][0]) > 1:
        for j in range(len(all_filtered_cons[i][0])):
            bb = cv.boundingRect(all_filtered_cons[i][0][j])
            iou = calc_iou(gt, bb)
            ious.append([iou, i])
    # Calculate IoU for single bounding box
    else:
        bb = cv.boundingRect(all_filtered_cons[i][0][0])
        iou = calc_iou(gt, bb)
        ious.append([iou, i])
    
    # print(gt)
    # print(bb)
    # print(iou)
    # img = cv.imread("test_data_10000/" + "+".join(time_stamp.split("-")) + "_infrared_image.png")
    # cv.rectangle(img, pt1=(bb[0], bb[1]), pt2=(bb[0]+bb[2], bb[1]+bb[3]), color=(0, 0, 255), thickness=2)
    # cv.rectangle(img, pt1=(gt[0], gt[1]), pt2=(gt[0]+gt[2], gt[1]+gt[3]), color=(0, 255, 0), thickness=2)
    # cv.imshow("test", img)
    # k = cv.waitKey(0)
    # cv.destroyWindow("test")
    # cv.waitKey(1)
    # if k == ord("q"):
    #     break

no_pred

7399

In [168]:
k = 0
for i in range(len(all_filtered_cons)):
    if len(all_filtered_cons[i][0]) >= 2:
        k += 1
k

30

In [185]:
print("### CONTOUR DETECTION MODEL RESULTS")
thres = [0.01, 0.5, 0.75]
for t in thres:
    true_pos = 0
    false_pos = 0
    false_neg = 0

    for i in range(len(all_filtered_cons)):
        if all_filtered_cons[i][0] == []:
            time_stamp = all_filtered_cons[i][1].split("_")[0]
            x = bb_df.loc[bb_df["timestamp"] == time_stamp, "x"].item()
            y = bb_df.loc[bb_df["timestamp"] == time_stamp, "y"].item()
            if not (x == 0 and y == 0):
                false_neg += 1
        elif len(all_filtered_cons[i][0]) == 1:
            for j in range(len(ious)):
                if ious[j][1] == i:
                    if ious[j][0] >= t:
                        true_pos += 1
                    if ious[j][0] < t:
                        false_pos += 1
                    break

    print("IoU threshold = ", t)
    print("\tTrue pos:\t", true_pos)
    print("\tFalse pos:\t", false_pos)
    print("\tFalse Neg:\t", false_neg)
    print("\tPrecision:\t", true_pos / (true_pos+false_pos))
    print("\tRecall:\t\t", true_pos / (true_pos+false_neg))



### CONTOUR DETECTION MODEL RESULTS
IoU threshold =  0.01
	True pos:	 760
	False pos:	 311
	False Neg:	 7370
	Precision:	 0.7096171802054155
	Recall:		 0.09348093480934809
IoU threshold =  0.5
	True pos:	 420
	False pos:	 651
	False Neg:	 7370
	Precision:	 0.39215686274509803
	Recall:		 0.05391527599486521
IoU threshold =  0.75
	True pos:	 236
	False pos:	 835
	False Neg:	 7370
	Precision:	 0.2203548085901027
	Recall:		 0.031028135682356035


### CONTOUR DETECTION MODEL RESULTS (most precise)
IoU threshold =  0.01<br>
&emsp;True pos:	 760<br>
&emsp;False pos:	 311<br>
&emsp;False Neg:	 7370<br>
&emsp;Precision:	 0.7096171802054155<br>
&emsp;Recall:		 0.09348093480934809<br>
<br>
IoU threshold =  0.5<br>
&emsp;True pos:	 420<br>
&emsp;False pos:	 651<br>
&emsp;False Neg:	 7370<br>
&emsp;Precision:	 0.39215686274509803<br>
&emsp;Recall:		 0.05391527599486521<br>
<br>
IoU threshold =  0.75<br>
&emsp;True pos:	 236<br>
&emsp;False pos:	 835<br>
&emsp;False Neg:	 7370<br>
&emsp;Precision:	 0.2203548085901027<br>
&emsp;Recall:		 0.031028135682356035<br>

### CONTOUR DETECTION MODEL RESULTS
IoU threshold =  0.01
<br>&emsp;Precision:&emsp;	 0.7096171802054155
<br>&emsp;Recall:&emsp;		 0.09348093480934809

<br><br>IoU threshold =  0.1
<br>&emsp;Precision:&emsp;	 0.6657329598506069
<br>&emsp;Recall:&emsp;		 0.08820982308548807

<br><br>IoU threshold =  0.5
<br>&emsp;Precision:&emsp;	 0.36134453781512604
<br>&emsp;Recall:&emsp;		 0.04989042155472476

<br><br>IoU threshold =  0.75
<br>&emsp;Precision:&emsp;	 0.19794584500466852
<br>&emsp;Recall:&emsp;		 0.02796096016882089